# Physical Object: Gate

The `Gate` class models a controllable gate in a water system. Its outflow is calculated using the orifice equation based on the upstream and downstream water levels and the area of the gate opening.

A key feature of this model is the `max_rate_of_change` parameter, which simulates the physical limitation of how quickly a gate can open or close. The gate's opening will ramp towards its target value (`control_signal`) at this limited rate.

### State Variables
- `opening` (float): The current opening of the gate (e.g., in meters or as a percentage).
- `outflow` (float): The calculated outflow through the gate (m^3/s).

### Parameters
- `discharge_coefficient` (float): A dimensionless factor C in the orifice equation.
- `width` (float): The width of the gate opening (m).
- `max_opening` (float): The maximum possible opening value.
- `max_rate_of_change` (float): The maximum speed at which the gate opening can change per second.

## Simulation Example

The following example simulates a gate being given a series of target openings. We can observe how the actual opening ramps towards the target and how this affects the outflow. Upstream and downstream water levels are held constant to isolate the gate's behavior.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from swp.simulation_identification.physical_objects.gate import Gate
from swp.core.interfaces import State, Parameters

# 1. Define initial state and parameters
initial_state = State(opening=0.1)
parameters = Parameters(
    discharge_coefficient=0.6,
    width=10.0,
    max_opening=1.0,
    max_rate_of_change=0.05 # Can only change by 5% per second
)

# 2. Create a Gate instance
gate = Gate(name="control_gate", initial_state=initial_state, parameters=parameters)

# 3. Simulation settings
dt = 1.0
simulation_duration = 100
num_steps = int(simulation_duration / dt)
history = []
target_openings = []

# 4. Run the simulation loop
for t in range(num_steps):
    # Create a varying control signal for the target opening
    if t < 20:
        target = 0.8
    elif t < 60:
        target = 0.2
    else:
        target = 1.0
    target_openings.append(target)
    
    action = {
        'control_signal': target,
        'upstream_head': 10.0, # Constant upstream water level
        'downstream_head': 8.0  # Constant downstream water level
    }
    
    current_state = gate.step(action=action, dt=dt)
    history.append(current_state.copy())

print("Gate simulation complete.")

## Results and Visualization

The plots show the `target_opening` (the control signal) versus the actual `opening` of the gate. The ramping behavior due to the `max_rate_of_change` is clearly visible. The `outflow` plot shows how the flow changes as a result of the gate's movement.

In [ ]:
# Extract data from history
time = [i * dt for i in range(num_steps)]
openings = [h['opening'] for h in history]
outflows = [h['outflow'] for h in history]

# Create a DataFrame
df = pd.DataFrame({
    'Time': time,
    'Target Opening': target_openings,
    'Actual Opening': openings,
    'Outflow': outflows
})

print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Gate Opening
ax1.plot(df['Time'], df['Target Opening'], label='Target Opening (Control Signal)', color='red', linestyle='--')
ax1.plot(df['Time'], df['Actual Opening'], label='Actual Opening', color='blue')
ax1.set_ylabel('Opening (%)')
ax1.set_title('Gate Dynamics')
ax1.grid(True)
ax1.legend()

# Outflow
ax2.plot(df['Time'], df['Outflow'], label='Outflow', color='green')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Outflow (m^3/s)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()